In [1]:
import matplotlib
%matplotlib ipympl
from matplotlib import pyplot as plt
import matplotlib.dates as mdates
from ipyleaflet import Map, Marker, MarkerCluster, WMSLayer, Unicode, basemaps
import numpy as np

wms = WMSLayer(
    url='https://basemap.nationalmap.gov/arcgis/rest/services/USGSImageryTopo/MapServer/tile/{z}/{y}/{x}',
    attribution='US Geological Survey'
)

m = Map(basemap=basemaps.CartoDB.Positron, center=(30.661, -88.645), zoom=5)
m.add_layer(wms)
m

Map(center=[30.661, -88.645], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoo…

In [2]:
fig0,ax0 = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [3]:
fig1,ax1 = plt.subplots()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [4]:
import requests,pandas,io
state_data=[]
for state in ['la','ms','ar','al','tn','mo']:
    resp = requests.post('https://waterservices.usgs.gov/nwis/site/?format=rdb&stateCd={0}&siteType=ST&siteStatus=active&hasDataTypeCd=iv,dv'.format(state))
    ncomments=0
    for line in resp.text.split('\n'):
        if '#' in line:
            ncomments+=1
        else:
            break
    state_data.append(pandas.read_table(io.BytesIO(resp._content),comment='#',skiprows=[ncomments+1]))
f = pandas.concat(state_data, axis=0)
f.head()

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,alt_datum_cd,huc_cd
0,USGS,2489500,"Pearl River near Bogalusa, LA",ST,30.793243,-89.820907,U,NAD83,54.64,1.00,NAVD88,3180004.0
1,USGS,2490193,"Pearl River at Pools Bluff near Bogalusa, LA",ST,30.705833,-89.846389,5,NAD83,0.32,0.16,NAVD88,3180004.0
2,USGS,2490200,Pearl River Navigation Canal Lock and Dam No. 3,ST,30.638056,-89.868056,5,NAD83,33.98,0.20,NAVD88,3180004.0
3,USGS,2491500,"Bogue Chitto River at Franklinton, LA",ST,30.842962,-90.162030,U,NAD83,123.81,0.01,NGVD29,3180005.0
4,USGS,2492000,"Bogue Chitto River near Bush, LA",ST,30.629356,-89.897296,U,NAD83,43.31,0.22,NAVD88,3180005.0


In [5]:
def plot_station_daily(site_no):
    import requests 
    import xml.etree.ElementTree as ET
    from zipfile import ZipFile
    from io import BytesIO
    import numpy as np
    # defining the api-endpoint  
    URL = "https://cida.usgs.gov/"
    API_ENDPOINT = "https://cida.usgs.gov/sediment/data"
    ogcfilter = ET.Element("ogc:Filter",attrib={"xmlns:ogc":"http://www.opengis.net/ogc"})
    property_is = ET.SubElement(ogcfilter, "ogc:PropertyIsEqualTo", attrib={"matchCase":"true"})
    ET.SubElement(property_is, "ogc:PropertyName").text = "SITE_NO"
    ET.SubElement(property_is, "ogc:Literal").text = '{0:08d}'.format(site_no)
    ogcfilter_string = ET.tostring(ogcfilter).decode()  
    data = {'email' : 'cekees@gmail.com',
            'format': 'tsv',
            'directDownload':'true',
            'dataTypes': 'sites_data_daily_discrete_',
            'dailyFilter' : ogcfilter_string,
            'discreteFilter' : ogcfilter_string
           }
    # sending post request and saving response as response object 
    search = requests.post(url = API_ENDPOINT, data = data)  
    download = requests.get(url=URL+search.text)
    zipfile = ZipFile(BytesIO(download._content))
    def get_comments(filename):
        ncomments=0
        comments=''
        with zipfile.open(filename) as f:
            commentline = f.readline().decode()
            while commentline:
                if commentline[0] == '#' or commentline[:2] == r'"#':
                    ncomments +=1
                    comments += commentline.strip(r'"').strip('#')
                    commentline=f.readline().decode()
                else:
                    commentline=None
        return (ncomments,comments)   
    if zipfile.filelist:
        ax0.clear()
        ax1.clear()
        try:
            (ncomments, comments) = get_comments('daily_data.tsv')
            f = pandas.read_table(zipfile.open('daily_data.tsv'),header=ncomments).dropna(axis=1,how='all')
            f.datetime = pandas.to_datetime(f.datetime)
            ax0.plot(f.datetime,f.DAILY_FLOW)
            ax1.scatter(f.DAILY_FLOW,f.DAILY_SSC)
            ax0.set_ylabel('Discharge [cfs]')
            ax0.set_xlabel('Date')
            ax1.set_xlabel('Discharge [cfs]')
            ax1.set_ylabel('Sediment(SSC) [mg/l]')
            ax1.set_title("Daily Sediment Data")
        except:
            #print("daily flow failed")
            (ncomments, comments) = get_comments('discrete_data.tsv')
            f = pandas.read_table(zipfile.open('discrete_data.tsv'),header=ncomments).dropna(axis=1,how='all')
            f.DATETIME = pandas.to_datetime(f.DATETIME)
            try:
                ax0.plot(f.DATETIME,f.INSTFLOW)
                #ax1.scatter(f.INSTFLOW,f.SSC)
            except:
                #print("inst flow failed")
                pass
            try:
                ax0.plot(f.DATETIME,f.DAILYFLOW)
                ssc = np.zeros(f.SSC.shape,'d')
                for i,ssci in enumerate(f.SSC):
                    if 'A' in ssci:
                        ssc[i] = float(ssci[2:])
                    else:
                        ssc[i] = float(ssci)
                p63 = np.zeros(f.P63.shape,'d')
                if 'P63' in f.columns:                
                    for i,p63i in enumerate(f.P63):
                        if type(p63i) is not float:
                            if 'A' in p63i:
                                p63[i] = float(p63i[2:])
                            else:
                                p63[i] = float(p63i)
                        else:
                            p63[i] = p63i
                ax1.scatter(f.DAILYFLOW,ssc)
                ax1.scatter(f.DAILYFLOW,p63)
            except:
                #print("daily flow data failed")
                pass
            ax0.set_ylabel('Discharge [cfs]')
            times = np.array(f.DATETIME)
            #ax0.set_title("Time Period {0}-{1}".format(str(f.DATETIME[0]),str(times[-1])))
            #ax0.get_xaxis().set_visible(False)
            ax1.set_xlabel('Discharge [cfs]')
            ax1.set_ylabel('Sediment(SSC) [mg/l]')
            ax1.set_title("Discrete Sediment Data")
    else:
        ax0.set_title("No Data")
        ax1.set_title("No Data")

In [6]:
f.head()

,agency_cd,site_no,station_nm,site_tp_cd,dec_lat_va,dec_long_va,coord_acy_cd,dec_coord_datum_cd,alt_va,alt_acy_va,alt_datum_cd,huc_cd
0,USGS,2489500,"Pearl River near Bogalusa, LA",ST,30.793243,-89.820907,U,NAD83,54.64,1.00,NAVD88,3180004.0
1,USGS,2490193,"Pearl River at Pools Bluff near Bogalusa, LA",ST,30.705833,-89.846389,5,NAD83,0.32,0.16,NAVD88,3180004.0
2,USGS,2490200,Pearl River Navigation Canal Lock and Dam No. 3,ST,30.638056,-89.868056,5,NAD83,33.98,0.20,NAVD88,3180004.0
3,USGS,2491500,"Bogue Chitto River at Franklinton, LA",ST,30.842962,-90.162030,U,NAD83,123.81,0.01,NGVD29,3180005.0
4,USGS,2492000,"Bogue Chitto River near Bush, LA",ST,30.629356,-89.897296,U,NAD83,43.31,0.22,NAVD88,3180005.0


In [7]:
stations = []
site_dict={}

def station_on_click_handler(event=None, id=None, properties=None, **args):
    plot_station_daily(site_dict[tuple(args['coordinates'])])

for site_no, station_nm, lat, long in zip(f.site_no,f.station_nm,f.dec_lat_va, f.dec_long_va):

    stations.append(Marker(location=(float(lat),float(long)), 
                           draggable=False,
                           title="Site: {0}\nStation: {1}".format(site_no,station_nm)))
    site_dict[tuple(stations[-1].location)] = site_no
    stations[-1].on_click(station_on_click_handler)

In [8]:
marker_cluster = MarkerCluster(
    markers=tuple(stations)
)

m.add_layer(marker_cluster);